In [21]:

# CREDIT: Followed guide here: https://huggingface.co/course/chapter7/5?fw=tf#creating-a-strong-baseline


# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nyt-news/nyt_all.csv
/kaggle/input/nyt-news/nyt_val.csv
/kaggle/input/nyt-news/nyt_train.csv
/kaggle/input/nyt-news/nyt_test.csv


In [174]:
#!rm -r /kaggle/working/t5_NytNews

In [23]:
from datasets import load_dataset
dataset = load_dataset('csv', data_files={'train':'/kaggle/input/nyt-news/nyt_train.csv','validation':'/kaggle/input/nyt-news/nyt_val.csv'},delimiter='~')
print(dataset.keys())

  0%|          | 0/2 [00:00<?, ?it/s]

dict_keys(['train', 'validation'])


In [24]:
def show_samples(dataset, num_samples=3, seed=42):
    sample = dataset["train"].shuffle(seed=seed).select(range(num_samples))
    for example in sample:
        print(f"\n'>> Title: {example['title']}'")
        print(f"'>> Article: {example['maintext']}'")


show_samples(dataset)


'>> Title: U.S. Proposal for Global Minimum Tax Gets Backing From 130 Countries'
'>> Article: The deal comes after four years of fraught international negotiations. If enacted, it would essentially stop countries from slashing their tax rates to lure businesses, a move that the United States and other high-tax jurisdictions say has deprived them of funding for crucial investments like infrastructure and education. Critical details still need to be worked out, including how to execute the plan, which is expected to be finalized in October, the economic cooperation organization said. Those details could also determine which American multinationals are subject to the new rules on digital taxation, which exclude financial services companies and those in extractive industries like oil and gas. And after that, new digital taxes and global corporate minimum taxes would still need to be approved by Congress and national legislatures. The organization estimates that over $100 billion of corpor

In [25]:
from transformers import AutoTokenizer

model_checkpoint = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, model_max_length=512)

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

In [26]:
#test tokenizer
inputs = tokenizer("MOSCOW — Russia on Thursday escalated a confrontation with Europe and the United States over the poisoning of a former Russian spy in Britain, saying it would expel 60 American diplomats and an unspecified number of envoys from other countries to retaliate for a mass expulsion of Russian diplomats working in the West and beyond that was ordered this week. Furious at what it described as an anti-Russian campaign orchestrated by Washington and London, the Kremlin exceeded an equivalent response to the United States and ordered the closing of the American Consulate in St. Petersburg, Russia’s second-largest city. The consulate is bigger and far more important to relations than the Russian Consulate in Seattle, which the Trump administration ordered closed on Monday as part of its expulsion decree. The crisis over the March 4 poisoning of a former Russian double agent and his daughter has driven tensions between the Kremlin and the West to their highest pitch in decades and forced European countries like Germany that are usually wary of clashing with Moscow to choose sides. Britain contends that the poison used was a signature Russian nerve agent created by Soviet-era scientists. Voicing alarm that the East-West confrontation was spinning out of control, the secretary general of the United Nations, António Guterres, said that the crisis recalled the Cold War, only without the controls and channels of communication established before the 1991 collapse of the Soviet Union “to make sure things would not get out of control when tensions rise.")
inputs

{'input_ids': [7109, 11151, 518, 3, 318, 4623, 30, 2721, 30254, 26, 3, 9, 25704, 28, 1740, 11, 8, 907, 1323, 147, 8, 14566, 53, 13, 3, 9, 1798, 4263, 14520, 16, 7190, 6, 2145, 34, 133, 1215, 4343, 1640, 797, 23661, 7, 11, 46, 73, 7576, 3676, 381, 13, 3, 35, 9845, 7, 45, 119, 1440, 12, 3, 60, 1947, 23, 342, 21, 3, 9, 3294, 1215, 20567, 13, 4263, 23661, 7, 464, 16, 8, 1244, 11, 1909, 24, 47, 5563, 48, 471, 5, 10759, 2936, 44, 125, 34, 3028, 38, 46, 1181, 18, 29613, 29, 2066, 13873, 1054, 57, 2386, 11, 1524, 6, 8, 14873, 51, 40, 77, 22080, 46, 7072, 1773, 12, 8, 907, 1323, 11, 5563, 8, 6733, 13, 8, 797, 11540, 342, 16, 472, 5, 25645, 6, 4623, 22, 7, 511, 18, 15599, 7, 17, 690, 5, 37, 6900, 14270, 19, 4038, 11, 623, 72, 359, 12, 5836, 145, 8, 4263, 11540, 342, 16, 8854, 6, 84, 8, 2523, 3602, 5563, 3168, 30, 2089, 38, 294, 13, 165, 1215, 20567, 28717, 5, 37, 5362, 147, 8, 1332, 314, 14566, 53, 13, 3, 9, 1798, 4263, 1486, 3102, 11, 112, 3062, 65, 6737, 7012, 7, 344, 8, 14873, 51, 40, 77, 11,

In [27]:
tokenizer.convert_ids_to_tokens(inputs.input_ids)

['▁MO',
 'SCO',
 'W',
 '▁',
 '—',
 '▁Russia',
 '▁on',
 '▁Thursday',
 '▁escalate',
 'd',
 '▁',
 'a',
 '▁confrontation',
 '▁with',
 '▁Europe',
 '▁and',
 '▁the',
 '▁United',
 '▁States',
 '▁over',
 '▁the',
 '▁poison',
 'ing',
 '▁of',
 '▁',
 'a',
 '▁former',
 '▁Russian',
 '▁spy',
 '▁in',
 '▁Britain',
 ',',
 '▁saying',
 '▁it',
 '▁would',
 '▁ex',
 'pel',
 '▁60',
 '▁American',
 '▁diplomat',
 's',
 '▁and',
 '▁an',
 '▁un',
 'spec',
 'ified',
 '▁number',
 '▁of',
 '▁',
 'en',
 'voy',
 's',
 '▁from',
 '▁other',
 '▁countries',
 '▁to',
 '▁',
 're',
 'tal',
 'i',
 'ate',
 '▁for',
 '▁',
 'a',
 '▁mass',
 '▁ex',
 'pulsion',
 '▁of',
 '▁Russian',
 '▁diplomat',
 's',
 '▁working',
 '▁in',
 '▁the',
 '▁West',
 '▁and',
 '▁beyond',
 '▁that',
 '▁was',
 '▁ordered',
 '▁this',
 '▁week',
 '.',
 '▁Fur',
 'ious',
 '▁at',
 '▁what',
 '▁it',
 '▁described',
 '▁as',
 '▁an',
 '▁anti',
 '-',
 'Russia',
 'n',
 '▁campaign',
 '▁orchestra',
 'ted',
 '▁by',
 '▁Washington',
 '▁and',
 '▁London',
 ',',
 '▁the',
 '▁Kre',
 'm',
 'l',
 

In [28]:
max_input_length = 512
max_target_length = 30


def preprocess_function(examples):
    model_inputs = tokenizer(
        examples["maintext"],
        max_length=max_input_length,
        truncation=True,
    )
    labels = tokenizer(
        examples["title"], max_length=max_target_length, truncation=True
    )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [29]:
tokenized_datasets = dataset.map(preprocess_function, batched=True)

  0%|          | 0/6 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

In [33]:
!pip install rouge_score
!pip install evaluate

In [34]:
import evaluate

rouge_score = evaluate.load("rouge")

In [12]:
!pip install nltk

In [13]:
import nltk

nltk.download("punkt")

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [16]:
from nltk.tokenize import sent_tokenize


def one_sentence_summary(text):
    return "\n".join(sent_tokenize(text)[:1])


print(one_sentence_summary(dataset["train"][1]["maintext"]))

Mr. Trump did not elaborate on the conversation.


In [17]:
print(dataset["train"][1]["maintext"])

Mr. Trump did not elaborate on the conversation. But he appeared to be referring to a phone call he had days earlier with King Salman. In his remarks here in Bahrain, home to the Navy’s Fifth Fleet, Mr. Mattis sought to tamp down increasing concerns that if Washington reduces its security role in the Middle East, Russia would move quickly to fill the vacuum. The United States has not had an aircraft carrier in the Persian Gulf since March, and the Pentagon is pulling Patriot missile-defense systems out of Jordan, Kuwait and Bahrain as part of a strategic realignment of firepower away from the Middle East and toward Russia and China. “Russia’s presence in the region cannot replace the longstanding, enduring and transparent U.S. commitment to the Middle East,” Mr. Mattis said. Indeed, the bulk of Mr. Mattis’s speech focused on what he said was Iran’s destabilizing actions in Iraq, Syria, Yemen and Lebanon — “spreading mischief and murder across the region,” as the secretary put it. The s

In [19]:
def evaluate_baseline(dataset, metric):
    summaries = [one_sentence_summary(text) for text in dataset["maintext"]]
    return metric.compute(predictions=summaries, references=dataset["title"])

In [190]:
import pandas as pd

score = evaluate_baseline(dataset["validation"], rouge_score)
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
rouge_dict = dict((rn, round(score[rn] * 100, 2)) for rn in rouge_names)
rouge_dict

{'rouge1': 13.91, 'rouge2': 3.47, 'rougeL': 11.62, 'rougeLsum': 11.63}

In [20]:
from transformers import TFAutoModelForSeq2SeqLM

model = TFAutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

2022-12-06 11:13:35.813645: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-06 11:13:35.814957: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-06 11:13:35.815718: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-06 11:13:35.818767: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

Downloading:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

2022-12-06 11:14:36.280959: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
All model checkpoint layers were used when initializing TFMT5ForConditionalGeneration.

All the layers of TFMT5ForConditionalGeneration were initialized from the model checkpoint at google/mt5-small.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMT5ForConditionalGeneration for predictions without further training.


In [192]:
from huggingface_hub import notebook_login

notebook_login()

In [193]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf")

In [194]:
tokenized_datasets = tokenized_datasets.remove_columns(
    dataset["train"].column_names
)

Another example of our tokenized dataset.

In [195]:
features = [tokenized_datasets["train"][i] for i in range(2)]
data_collator(features)

{'input_ids': <tf.Tensor: shape=(2, 457), dtype=int32, numpy=
array([[   441,   1906,  73353,   9057,  18737,    852, 123810,    336,
          2477,    293,    270,    783,    259,    262,   2718,  31499,
           260,   4837,    787,   1002,    351,    259,   3824,   2952,
         12747,    263,    260,   1306,   1627,   6583,    304,    259,
        154823,   1406,   3230,   3026,    287,    259,  84139,   9195,
         13050,    261,  21825,    304,    287,   9195,  23473,    261,
          3026,   1525,  14313,    261,   1532,   1150,   3026,  28210,
           260,   1829,    787,   8448,   2559,    514,    259,    438,
           261,    305,    609,    293,    263,    259,    262,   3230,
           288,    259,  85109,   2559,    288,   1546,   7868,    260,
          1385,    293,    263,   1388,   6557,    347,    288,    342,
           527,   2829,    527,    787,    738,    281,   1371,   3117,
           261,   1156,    281, 164992,    264,  94142,  42489,    263,
  

In [196]:
tf_train_dataset = model.prepare_tf_dataset(
    tokenized_datasets["train"],
    collate_fn=data_collator,
    shuffle=True,
    batch_size=8,
)
tf_eval_dataset = model.prepare_tf_dataset(
    tokenized_datasets["validation"],
    collate_fn=data_collator,
    shuffle=False,
    batch_size=8,
)


Defining optimizer

In [197]:
from transformers import create_optimizer
import tensorflow as tf

# The number of training steps is the number of samples in the dataset, divided by the batch size then multiplied
# by the total number of epochs. Note that the tf_train_dataset here is a batched tf.data.Dataset,
# not the original Hugging Face Dataset, so its len() is already num_samples // batch_size.
num_train_epochs = 8
num_train_steps = len(tf_train_dataset) * num_train_epochs
model_name = model_checkpoint.split("/")[-1]

optimizer, schedule = create_optimizer(
    init_lr=5.6e-5,
    num_warmup_steps=0,
    num_train_steps=num_train_steps,
    weight_decay_rate=0.01,
)

model.compile(optimizer=optimizer,)

# Train in mixed-precision float16
tf.keras.mixed_precision.set_global_policy("mixed_float16")

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


# Training model on our data set

In [198]:
!mkdir -p mt5_NytNews

In [199]:
!apt-get install git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.9.2-1).
0 upgraded, 0 newly installed, 0 to remove and 93 not upgraded.


In [200]:
from transformers.keras_callbacks import PushToHubCallback

callback = PushToHubCallback(
    output_dir=f"/kaggle/working/mt5_NytNews", tokenizer=tokenizer
)

model.fit(
    tf_train_dataset, validation_data=tf_eval_dataset, callbacks=[callback], epochs=8
)

/opt/conda/lib/python3.7/site-packages/huggingface_hub/repository.py:708: FutureWarning: Creating a repository through 'clone_from' is deprecated and will be removed in v0.11.
  FutureWarning,
Cloning https://huggingface.co/Matthewww/mt5_NytNews into local empty directory.


Epoch 1/8


/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/indexed_slices.py:442: UserWarning: Converting sparse IndexedSlices to a dense Tensor with 128057344 elements. This may consume a large amount of memory.
  num_elements)


750/750 [==============================] - 271s 320ms/step - loss: nan - val_loss: nan


Adding files tracked by Git LFS: ['tokenizer.json']. This may take a bit of time if the files are large.


Epoch 2/8
750/750 [==============================] - 245s 326ms/step - loss: nan - val_loss: nan
Epoch 3/8
750/750 [==============================] - 234s 312ms/step - loss: nan - val_loss: nan
Epoch 4/8
750/750 [==============================] - 234s 312ms/step - loss: nan - val_loss: nan
Epoch 5/8
750/750 [==============================] - 233s 311ms/step - loss: nan - val_loss: nan
Epoch 6/8
750/750 [==============================] - 234s 311ms/step - loss: nan - val_loss: nan
Epoch 7/8
750/750 [==============================] - 234s 312ms/step - loss: nan - val_loss: nan
Epoch 8/8
750/750 [==============================] - 234s 312ms/step - loss: nan - val_loss: nan


The loss calculated above is pretty uselss, keras didn't seem to have an apporpiate loss function available by default.

In [201]:
!pip install tqdm

In [205]:
from tqdm import tqdm
import numpy as np

generation_data_collator = DataCollatorForSeq2Seq(
    tokenizer, model=model, return_tensors="tf", pad_to_multiple_of=320
)

tf_generate_dataset = model.prepare_tf_dataset(
    tokenized_datasets["validation"],
    collate_fn=generation_data_collator,
    shuffle=False,
    batch_size=8,
    drop_remainder=True,
)


@tf.function(jit_compile=True)
def generate_with_xla(batch):
    return model.generate(
        input_ids=batch["input_ids"],
        attention_mask=batch["attention_mask"],
        max_new_tokens=32,
    )


all_preds = []
all_labels = []
for batch, labels in tqdm(tf_generate_dataset):
    predictions = generate_with_xla(batch)
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = labels.numpy()
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    decoded_preds = ["\n".join(sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(sent_tokenize(label.strip())) for label in decoded_labels]
    all_preds.extend(decoded_preds)
    all_labels.extend(decoded_labels)

100%|██████████| 375/375 [01:24<00:00,  5.38it/s]Exception ignored in: <function IteratorResourceDeleter.__del__ at 0x7f4564af9ef0>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/tensorflow/python/data/ops/iterator_ops.py", line 546, in __del__
    handle=self._handle, deleter=self._deleter)
  File "/opt/conda/lib/python3.7/site-packages/tensorflow/python/ops/gen_dataset_ops.py", line 1264, in delete_iterator
    _ctx, "DeleteIterator", name, handle, deleter)
KeyboardInterrupt: 
100%|██████████| 375/375 [02:21<00:00,  2.64it/s]


Trying to calculate Rouge scores.....

In [211]:
result = rouge_score.compute(
    predictions=decoded_preds, references=decoded_labels, use_stemmer=True
)
print(result)

{'rouge1': 0.0, 'rouge2': 0.0, 'rougeL': 0.0, 'rougeLsum': 0.0}


In [212]:
from transformers import pipeline

hub_model_id = "Matthewww/mt5_NytNews"
summarizer = pipeline("summarization", model=hub_model_id)

Downloading:   0%|          | 0.00/745 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFMT5ForConditionalGeneration.

All the layers of TFMT5ForConditionalGeneration were initialized from the model checkpoint at Matthewww/mt5_NytNews.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMT5ForConditionalGeneration for predictions without further training.


Downloading:   0%|          | 0.00/455 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.11M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/15.6M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/74.0 [00:00<?, ?B/s]

# Testing out the model!

In [222]:
def print_summary(idx):
    review = dataset["validation"][idx]["maintext"]
    title = dataset["validation"][idx]["title"]
    summary = summarizer(dataset["validation"][idx]["maintext"])[0]["summary_text"]
    print(f"'>>> Article: {review}'")
    print(f"\n'>>> Title: {title}'")
    print(f"\n'>>> Generated Headline: {summary}'")
    
print_summary(100)

'>>> Article: JERUSALEM — Benjamin Netanyahu was poised to return to power. But there was a cloud over his apparent turnaround, the result of an increasingly shrill campaign that raised questions about his ability to heal Israel’s internal wounds or better its standing in the world. He said there would be no Palestinian state under his watch. He railed against Israeli Arabs — because they had gone out to vote. From the capitals of Europe, to Washington, to the West Bank, to the streets of Israel, even while his critics said Mr. Netanyahu had reaffirmed his reputation as a cynical, calculating politician, it appeared that his approach succeeded in drawing votes from other right-leaning parties. But along the way he angered the president of the United States with a speech to Congress and infuriated European leaders eager to see the peace process move ahead to create a Palestinian state.'

'>>> Title: Deep Wounds and Lingering Questions After Israel’s Bitter Race'

'>>> Generated Headline

In [221]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("Matthewww/mt5_NytNews", from_tf=True)
model = AutoModelForSeq2SeqLM.from_pretrained("Matthewww/mt5_NytNews", from_tf=True)

device = torch.device("cpu")
model = model.to(device)
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("Matthewww/mt5_NytNews", from_tf=True)
model = AutoModelForSeq2SeqLM.from_pretrained("Matthewww/mt5_NytNews", from_tf=True)

device = torch.device("cpu")
model = model.to(device)

text = """"
The World Cup has historically been a boon for Twitter, bringing in record traffic and an influx of advertising dollars.

But this time, when the global soccer tournament started on Nov. 20, Twitter’s U.S. ad revenue was running at 80 percent below internal expectations for that week, three people with knowledge of the figures said.

In tandem, Twitter was rapidly cutting its revenue projections. The company previously forecast that it would generate $1.4 billion in the last three months of the year, down from $1.6 billion a year ago because of the global economic downturn. But as Twitter kept missing its weekly advertising targets, that number slid to $1.3 billion, then to $1.1 billion, two people said.

Elon Musk, Twitter’s new owner, has warned repeatedly that his social media company faces dire financial straits. Interviews with seven former employees and internal documents seen by The New York Times paint a fuller picture of Twitter’s financial woes.
"""

preprocess_text = text.strip().replace("\n","")
tokenized_text = tokenizer.encode(preprocess_text, return_tensors="pt").to(device)

summary_ids = model.generate(
            tokenized_text,
            max_length=50,
            num_beams=2,
            repetition_penalty=2.5,
            length_penalty=1.0,
            early_stopping=True
        )

output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

print(output)

All TF 2.0 model weights were used when initializing MT5ForConditionalGeneration.

Some weights of MT5ForConditionalGeneration were not initialized from the TF 2.0 model and are newly initialized: ['decoder.embed_tokens.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
All TF 2.0 model weights were used when initializing MT5ForConditionalGeneration.

Some weights of MT5ForConditionalGeneration were not initialized from the TF 2.0 model and are newly initialized: ['decoder.embed_tokens.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<extra_id_0> to $1.3 billion


The estimation is just a random three 'words' from the article, not very good...maybe some issues with training the model.